- # Integração Arduino com Python
***

- #### Add from Arduino import Arduino into your python script to communicate with your Arduino.

In [1]:
from Arduino import Arduino
import time

***
- ## Definindo variáveis

In [2]:
green_led, red_led = 13, 7
g, r = 1, 1
detector_pin = 3

***
- ## void setup()

In [3]:
board = Arduino("9600", port="/dev/ttyACM0") # plugged in via USB, serial com at rate 9600

board.pinMode(detector_pin, "INPUT") # Configures DETECTOR pin as input
board.pinMode(green_led, "OUTPUT") # Configures LED pin as output
board.pinMode(red_led, "OUTPUT") # Configures LED pin as output


You need to update the version of the Arduino-Python3 library running on your Arduino.
The Arduino sketch is version
The Python installation is V0.6
Flash the prototype sketch again.


***

- ## void loop()

In [4]:
while True:

    if (board.digitalRead(detector_pin)):
        if g == 1:
            board.digitalWrite(green_led, "LOW")
            g *= -1
        if r == -1:
            board.digitalWrite(red_led, "HIGH")
            r *= -1
    else:
        if r == 1:
            board.digitalWrite(red_led, "LOW")
            r *= -1
        if g == -1:
            board.digitalWrite(green_led, "HIGH")
            g *= -1


KeyboardInterrupt: 

In [2]:
import time
from datetime import datetime
today_date_time = datetime.today()
print(str(today_date_time))

2023-01-27 10:32:29.225390


In [6]:
sensor_id = 'AUPS1' # arduino-uno-presence_sensor-1
presence_id = str(today_date_time) + sensor_id
print(presence_id)

31
